In this project, we'll build a simple convolutional neural network for CIFAR-10 image classification. Code contained in this project was based on Tensorflow 1.2.1 and python 3.5. 

In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import tarfile
import tensorflow as tf
import helper
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

cifar10_dataset_folder_path = 'cifar-10-batches-py'
tar_gz_path = 'cifar-10-python.tar.gz'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(tar_gz_path):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            tar_gz_path,
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open(tar_gz_path) as tar:
        tar.extractall()
        tar.close()

# Preprocess Data

## Normalize

In [2]:
def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    """
    batch_mean = np.mean(x)
    batch_std = np.std(x)
    x = x.astype(np.float32)
    for ii in range(x.shape[0]):
        x[ii,:,:,:] = (x[ii,:,:,:] - batch_mean) / batch_std
    return x

## One hot encode

In [3]:
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """
    smooth_factor = 0.1
    output = np.zeros([len(x), 10]) # +( smooth_factor / 9 )
    for idx, item in enumerate(x):
        output[idx, item] = 1. #- smooth_factor
    return output

This above function is equivalent to tf.one_hot(x, 10), but tensorflow module can not be pickled so we're sticking with the above implementation.

## Preprocess Data
We will randomly shuffle the data, normalize them and save them in binary format.

In [4]:
import tensorflow as tf
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(cifar10_dataset_folder_path, one_hot_encode)

# Checkpoint
The above work is all saved so when we're revisiting this notebook we don't have to do those work again. We can start from here.

In [5]:
import pickle
import helper

valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))
valid_features = np.transpose(valid_features,(0, 3, 1, 2))
valid_features = normalize(valid_features)

# Building the network

## Defining input

In [6]:
def neural_net_image_input(image_shape):
    return tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], image_shape[2]], "x")


def neural_net_label_input(n_classes):
    return tf.placeholder(tf.float32, [None, n_classes], "y")


def neural_net_keep_prob_input(n):
    return tf.placeholder(tf.float32, None, "keep_prob_"+str(n))


def neural_net_training_flag():
    return tf.placeholder(tf.bool, None, "train_flag")

## Convolution and maxpool layer

In [7]:
def conv2d(x_tensor, conv_num_outputs, conv_ksize, conv_strides, is_train):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    w = tf.get_variable("w", shape=[conv_ksize[0], conv_ksize[1], x_tensor.get_shape().as_list()[1], conv_num_outputs],
                         initializer=tf.contrib.layers.xavier_initializer())
    
    #b = tf.Variable(tf.truncated_normal([conv_num_outputs],
    #                                  mean=0.0, stddev=0.1, dtype=tf.float32))
    
    wc = tf.nn.conv2d(x_tensor, w, strides=[1, conv_strides[0], conv_strides[1], 1], padding='SAME', data_format='NCHW')
    # z = tf.nn.bias_add(wc, b)
    z = tf.layers.batch_normalization(wc, training=is_train)
    
    return tf.nn.relu(z)



## Flatten layer
Implement the flatten function to change the dimension of x_tensor from a 4-D tensor to a 2-D tensor. The output should be the shape (Batch Size, Flattened Image Size). Shortcut option: you can use classes from the TensorFlow Layers or TensorFlow Layers (contrib) packages for this layer. For more of a challenge, only use other TensorFlow packages.

In [8]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    # reference : https://github.com/tensorflow/tensorflow/issues/7253
    return tf.reshape(x_tensor, [tf.shape(x_tensor)[0], np.prod(x_tensor.get_shape().as_list()[1:])])
    
    # This also works
    #return tf.reshape(x_tensor, [-1, np.prod(x_tensor.shape[1:]).value])

## Fully connected layer

In [9]:
def fully_conn(x_tensor, num_outputs, is_train):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    fc = tf.reshape(x_tensor, [-1, np.prod(x_tensor.get_shape().as_list()[1:])])
    
    w = tf.get_variable("w", shape=[np.prod(x_tensor.get_shape().as_list()[1:]), num_outputs],
                         initializer=tf.contrib.layers.xavier_initializer())
    
    # b = tf.Variable(tf.truncated_normal([num_outputs],mean=0.0, stddev=0.1, dtype=tf.float32))
    z = tf.matmul(fc, w)
    z = tf.layers.batch_normalization(z, training=is_train)
    
    return tf.nn.relu(z)

## Output layer

In [10]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    w = tf.get_variable("w", shape=[np.prod(x_tensor.get_shape().as_list()[1:]), num_outputs],
                         initializer=tf.contrib.layers.xavier_initializer())
    
    b = tf.Variable(tf.truncated_normal([num_outputs],mean=0.0, stddev=0.1, dtype=tf.float32))
    return tf.add(tf.matmul(x_tensor, w), b)


## Create the convolutional neural network model

In [11]:
def conv_net(x, keep_prob_1, keep_prob_2, keep_prob_3, keep_prob_4,  train_flag):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    
    # Convolution and maxpooling layers
    with tf.variable_scope("conv1"):
        conv = conv2d(x, 64, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_1)
    with tf.variable_scope("conv2"):
        conv = conv2d(conv, 64, (3, 3), (1, 1), train_flag)
    conv = tf.nn.max_pool(conv, ksize=[1, 1, 2, 2], strides=[1, 1, 2, 2], padding='SAME', data_format='NCHW')
    conv = tf.nn.dropout(conv, keep_prob_1)
    
    with tf.variable_scope("conv3"):
        conv = conv2d(conv, 128, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_2)
    with tf.variable_scope("conv4"):
        conv = conv2d(conv, 128, (3, 3), (1, 1), train_flag)
    conv = tf.nn.max_pool(conv, ksize=[1, 1, 2, 2], strides=[1, 1, 2, 2], padding='SAME', data_format='NCHW')
    conv = tf.nn.dropout(conv, keep_prob_2)
    
    with tf.variable_scope("conv5"):
        conv = conv2d(conv, 256, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_3)
    with tf.variable_scope("conv6"):
        conv = conv2d(conv, 256, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_3)
    with tf.variable_scope("conv7"):
        conv = conv2d(conv, 256, (3, 3), (1, 1), train_flag)
    conv = tf.nn.max_pool(conv, ksize=[1, 1, 2, 2], strides=[1, 1, 2, 2], padding='SAME', data_format='NCHW')
    conv = tf.nn.dropout(conv, keep_prob_3)
    
    with tf.variable_scope("conv8"):
        conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_4)
    with tf.variable_scope("conv9"):
        conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_4)
    with tf.variable_scope("conv10"):
        conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    conv = tf.nn.max_pool(conv, ksize=[1, 1, 2, 2], strides=[1, 1, 2, 2], padding='SAME', data_format='NCHW')
    conv = tf.nn.dropout(conv, keep_prob_4)
    
    with tf.variable_scope("conv11"):
        conv = conv2d(conv, 512, (1, 1), (1, 1), train_flag)
    conv = tf.nn.dropout(conv, keep_prob_4)
    
    
    #with tf.variable_scope("conv11"):
    #    conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    #conv = tf.nn.dropout(conv, keep_prob_4)
    #with tf.variable_scope("conv12"):
    #    conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    #conv = tf.nn.dropout(conv, keep_prob_4)
    #with tf.variable_scope("conv13"):
    #    conv = conv2d(conv, 512, (3, 3), (1, 1), train_flag)
    #conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #conv = tf.nn.dropout(conv, keep_prob_4)
    
    
    # Flatten Layer
    f = flatten(conv)

    # Fully Connected layers
    with tf.variable_scope("fc1"):
        fc = fully_conn(f, 512, train_flag)
        fc = tf.nn.dropout(fc, keep_prob_4)
    with tf.variable_scope("fc2"):
        fc = fully_conn(fc, 512, train_flag)
        fc = tf.nn.dropout(fc, keep_prob_4)
    #with tf.variable_scope("fc3"):
    #    fc = fully_conn(fc, 512, train_flag)
    #    fc = tf.nn.dropout(fc, keep_prob_4)
    
    # Output Layer
    with tf.variable_scope("out"):
        o = output(fc, 10)
    
    # TODO: return output
    return o


## Build the network

In [12]:
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((3, 32, 32))
y = neural_net_label_input(10)
keep_prob_1 = neural_net_keep_prob_input(1)
keep_prob_2 = neural_net_keep_prob_input(2)
keep_prob_3 = neural_net_keep_prob_input(3)
keep_prob_4 = neural_net_keep_prob_input(4)
train_flag = neural_net_training_flag()

# Model
logits = conv_net(x, keep_prob_1, keep_prob_2, keep_prob_3, keep_prob_4, train_flag)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Collect batch mean and variance for batch normalization
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

## Train network

In [13]:
def train_neural_network(session, optimizer, kp1, kp2, kp3, kp4, feature_batch, label_batch, is_train):
    """
    Optimize the session on a batch of images and labels
    """
    session.run(optimizer, feed_dict={x: feature_batch, 
                                      y: label_batch, 
                                      keep_prob_1: kp1,
                                      keep_prob_2: kp2,
                                      keep_prob_3: kp3,
                                      keep_prob_4: kp4,
                                      train_flag:is_train})

## Print Stats
It's important to evaluate the performance of model once in a while. If effect, we're feeding a small batch of data to the neural network through forward propagation and then caculate the accuracy of prediction. We don't want to do this too often as this slows down the overall process. It's important to keep in mind that since we're actually using the model for prediction but not training it, we need to set keep probability for dropout to 1 so we're not losing any connection between neurons. 

In [14]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    """
    loss = session.run(cost, feed_dict={x: feature_batch, 
                                        y: label_batch, 
                                        keep_prob_1: 1., 
                                        keep_prob_2: 1.,
                                        keep_prob_3: 1.,
                                        keep_prob_4: 1.,
                                        train_flag:False})
    valid_acc = session.run(accuracy, feed_dict={x: valid_features, 
                                                 y: valid_labels, 
                                                 keep_prob_1: 1., 
                                                 keep_prob_2: 1.,
                                                 keep_prob_3: 1.,
                                                 keep_prob_4: 1.,
                                                 train_flag:False})
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, valid_acc))

## Hyperparameters

In [15]:
epochs = 2000
batch_size = 64
kp1 = 0.8
kp2 = 0.7
kp3 = 0.6
kp4 = 0.5

## Train on a single CIFAR-10 batch

In [16]:
#print('Checking the Training on a Single Batch...')
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())
    
    # Training cycle
#    for epoch in range(epochs):
#        batch_i = 1
#        for batch_features, batch_labels in helper.load_preprocess_training_batch(batch_i, batch_size):
#            train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels, True)
#        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
#        print_stats(sess, batch_features, batch_labels, cost, accuracy)

## Fully train the model
Deep structure 11 layers, withFliplr, Gaussian Blur, Crop, and Affine, 200 Epochs : 91.1 %

In [18]:
import time
import helper
import imgaug as ia
from imgaug import augmenters as iaa

sometimes = lambda aug: iaa.Sometimes(0.3, aug)


save_model_path = './image_classification'


print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = 5
        index = np.array(range(n_batches)) + 1
        np.random.shuffle(index)
        
        for batch_i in index: # range(1, n_batches + 1):
            
            for batch_features, batch_labels in helper.load_preprocess_training_batch(batch_i, batch_size):
                
                seq = iaa.Sequential([iaa.Fliplr(0.5),
                                      iaa.Flipud(0.2),
                                      sometimes(iaa.Crop(px=2)),
                                      sometimes(iaa.Affine(scale={"X":(0.8,1.2), "y":(0.8,1.2)}, 
                                                           rotate=(-45,45), shear=(-16,16),
                                                           translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                                           cval=(0,255), mode=ia.ALL)),
                                      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.5)),
                                      sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)),
                                      sometimes(iaa.GaussianBlur(sigma=(0, 3.0)))
                                     ], random_order=True)
                batch_features = seq.augment_images(batch_features)
                
                batch_features = normalize(batch_features)
                
                batch_features = np.transpose(batch_features, (0, 3, 1, 2))   
                
                train_neural_network(sess, optimizer, kp1, kp2, kp3, kp4, batch_features, batch_labels, True)
            
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        print_stats(sess, batch_features, batch_labels, cost, accuracy)
        
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Training...
Epoch  1, CIFAR-10 Batch 1:  Loss:     2.0559 Validation Accuracy: 0.195400
training one batch took:62.4124 seconds
Epoch  2, CIFAR-10 Batch 5:  Loss:     2.0536 Validation Accuracy: 0.227400
training one batch took:57.3689 seconds
Epoch  3, CIFAR-10 Batch 1:  Loss:     1.9129 Validation Accuracy: 0.266200
training one batch took:57.6723 seconds
Epoch  4, CIFAR-10 Batch 1:  Loss:     1.8352 Validation Accuracy: 0.282200
training one batch took:58.2500 seconds
Epoch  5, CIFAR-10 Batch 1:  Loss:     1.9180 Validation Accuracy: 0.330800
training one batch took:58.8352 seconds
Epoch  6, CIFAR-10 Batch 5:  Loss:     1.4686 Validation Accuracy: 0.354400
training one batch took:58.5081 seconds
Epoch  7, CIFAR-10 Batch 2:  Loss:     1.7625 Validation Accuracy: 0.396800
training one batch took:58.2588 seconds
Epoch  8, CIFAR-10 Batch 3:  Loss:     1.5576 Validation Accuracy: 0.404000
training one batch took:58.4078 seconds
Epoch  9, CIFAR-10 Batch 1:  Loss:     1.5545 Validation Acc

KeyboardInterrupt: 

## Test model

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import pickle
import helper
import random

batch_size = 64

save_model_path = './image_classification'
n_samples = 4
top_n_predictions = 3

def test_model():
    """
    Test the saved model against the test dataset
    """
    test_features, test_labels = pickle.load(open('preprocess_test.p', mode='rb'))
    loaded_graph = tf.Graph()
    test_features = np.transpose(test_features,(0, 3, 1, 2))
    test_features = normalize(test_features)
    
    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob_1 = loaded_graph.get_tensor_by_name('keep_prob_1:0')
        loaded_keep_prob_2 = loaded_graph.get_tensor_by_name('keep_prob_2:0')
        loaded_keep_prob_3 = loaded_graph.get_tensor_by_name('keep_prob_3:0')
        loaded_keep_prob_4 = loaded_graph.get_tensor_by_name('keep_prob_4:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        loaded_train_flag = loaded_graph.get_tensor_by_name('train_flag:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for test_feature_batch, test_label_batch in helper.batch_features_labels(test_features, test_labels, batch_size):
            
            test_batch_acc_total += sess.run(loaded_acc,
                feed_dict={loaded_x: test_feature_batch, loaded_y: test_label_batch, 
                           loaded_keep_prob_1: 1.0, loaded_keep_prob_2: 1.0,
                           loaded_keep_prob_3: 1.0, loaded_keep_prob_4: 1.0,
                           loaded_train_flag:False})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))

        # Print Random Samples
        random_test_features, random_test_labels = tuple(zip(*random.sample(list(zip(test_features, test_labels)), n_samples)))
        
        random_test_predictions = sess.run(tf.nn.top_k(tf.nn.softmax(loaded_logits), top_n_predictions),
            feed_dict={loaded_x: random_test_features, loaded_y: random_test_labels, 
                       loaded_keep_prob_1: 1.0, loaded_keep_prob_2: 1.0,
                        loaded_keep_prob_3: 1.0, loaded_keep_prob_4: 1.0,
                       loaded_train_flag:False})
        helper.display_image_predictions(random_test_features, random_test_labels, random_test_predictions)

test_model()

2x64, 2x128, 3x256, 6x512(keep_prob:0.6), 5 max pooling layer, 300 epochs : 91.1%  
2x64, 2x128, 3x256, 6x512, 4 max pooling layer, 300 epochs : 90.9%

14 layers, Flipr(0.5), Flipud(0.2), 0.3(crop(2)), 0.3(Affine), 0.3(Gaussian(0,3)), 0.3(GaussianNoise(0,0.05*255),per channel 0.5), 0.3(Multiply(0.8,1.2), 400 epochs: 92%, 1200 epochs : 92.8%, 2000 epochs:93.1%